In [17]:
from pulp import *
import pandas as pd

def maximize_vorp(players, roster_size, min_qbs, max_qbs, min_rbs, max_rbs, min_wrs, max_wrs, min_tes, max_tes,
                  starting_qb, starting_rb, starting_wr, starting_te, starting_flex, picks, available, adp):
    # Initialize model
    model = LpProblem("FantasyFootballOptimization", LpMaximize)
    
    # Decision Variables: Binary variables indicating if a player is selected and/or starting
    players_selected = LpVariable.dicts("PlayerSelected", players, cat='Binary')
    players_starting = LpVariable.dicts("PlayerStarting", players, cat='Binary')
    
    # Objective Function: Maximize total VORP, with a bonus for starting players
    model += lpSum([vorp[player] * players_selected[player] for player in players]) \
            + lpSum([vorp[player] * 2 * players_starting[player] for player in players]), "TotalVORP"
    
    # Constraints
    # Roster size constraint
    model += lpSum([players_selected[player] for player in players]) == roster_size
    
    # Positional constraints: Minimum and maximum number of players at each position
    model += lpSum([players_selected[player] for player in players if player_position[player] == "QB"]) >= min_qbs
    model += lpSum([players_selected[player] for player in players if player_position[player] == "QB"]) <= max_qbs
    model += lpSum([players_selected[player] for player in players if player_position[player] == "RB"]) >= min_rbs
    model += lpSum([players_selected[player] for player in players if player_position[player] == "RB"]) <= max_rbs
    model += lpSum([players_selected[player] for player in players if player_position[player] == "WR"]) >= min_wrs
    model += lpSum([players_selected[player] for player in players if player_position[player] == "WR"]) <= max_wrs
    model += lpSum([players_selected[player] for player in players if player_position[player] == "TE"]) >= min_tes
    model += lpSum([players_selected[player] for player in players if player_position[player] == "TE"]) <= max_tes
    
    # Starting lineup constraints: Number of players at each position
    model += lpSum([players_starting[player] for player in players if player_position[player] == "QB"]) == starting_qb
    model += lpSum([players_starting[player] for player in players if player_position[player] == "RB"]) >= starting_rb
    model += lpSum([players_starting[player] for player in players if player_position[player] == "WR"]) >= starting_wr
    model += lpSum([players_starting[player] for player in players if player_position[player] == "TE"]) >= starting_te
    # Flex position constraint
    model += lpSum([players_starting[player] for player in players if player_position[player] in ["RB", "WR","TE"]]) == \
        starting_wr + starting_rb + starting_te + starting_flex

    # Constraint: Any player starting must also be selected
    for player in players:
        model += players_starting[player] <= players_selected[player]
    
    # Constraint: For each pick, we can only select players with an ADP greater than or equal to the pick and not already taken
    for pick in picks:
        available_players = available[pick-picks[0]:]
        model += lpSum([players_selected[player] for player in available_players]) >= len(picks) - picks.index(pick)
        
    # Solve the problem
    model.solve()
    
    # Extract results
    selected_players = [player for player in players if players_selected[player].value() == 1]
    starting_players = [player for player in players if players_starting[player].value() == 1]
    total_vorp = value(model.objective)
    
    return selected_players, starting_players, total_vorp

if __name__ == "__main__":
    # Load data
    df = pd.read_csv('vorp2023.csv')
    # get into the right format
    df['Player'] = df['FirstName'] + ' ' + df['LastName']
    players = df['Player'].tolist()
    vorp = df.set_index('Player')['Y0_VORP'].to_dict()
    player_position = df.set_index('Player')['Position'].to_dict()
    ADP = df.set_index('Player')['RedraftHalfPPR'].to_dict()
    
    # Constraints
    min_qbs, max_qbs = 1, 3
    min_rbs, max_rbs = 2, 6
    min_wrs, max_wrs = 2, 6
    min_tes, max_tes = 1, 2
    starting_qb = 1
    starting_rb = 2
    starting_wr = 2
    starting_te = 1
    starting_flex = 1
    roster_size = 13
    
    available = players[8:]
    teams = 12
    picks = []
    pick = 9
    for i in range(1,roster_size+1):
        for j in range(1,teams+1):
            if j == pick and i % 2 == 1:
                picks.append(((i-1)*teams)+j)
            elif j == teams - pick + 1 and i % 2 == 0:
                picks.append(((i-1)*teams)+j)
    
    # Constraints
    min_qbs, max_qbs = 1, 3
    min_rbs, max_rbs = 2, 6
    min_wrs, max_wrs = 2, 6
    min_tes, max_tes = 1, 2
    starting_qb = 1
    starting_rb = 2
    starting_wr = 2
    starting_te = 1
    starting_flex = 1
    
    # Run optimization
    selected_players, starting_players, total_vorp = maximize_vorp(players, roster_size, min_qbs, max_qbs, min_rbs, max_rbs,
                                                                   min_wrs, max_wrs, min_tes, max_tes, starting_qb, starting_rb,
                                                                   starting_wr, starting_te, starting_flex, picks, available, ADP)
    
    # Output results, showing each pos, player, vorp, and adp sorted by adp
    selected_players = sorted(selected_players, key=lambda x: ADP[x])
    print('Selected Players')
    for player in selected_players:
        print(player, player_position[player], ADP[player])
        
    print('\nStarting Players')
    starting_players = sorted(starting_players, key=lambda x: ADP[x])
    for player in starting_players:
        print(player, player_position[player], ADP[player])
        


9 ['Saquon Barkley', 'Stefon Diggs', 'Nick Chubb', 'AJ Brown', 'CeeDee Lamb', 'Patrick Mahomes', 'Derrick Henry', 'Davante Adams', 'Jonathan Taylor', 'Amon-Ra St Brown', 'Josh Jacobs', 'Josh Allen', 'Tony Pollard', 'Garrett Wilson', 'Jaylen Waddle', 'Jalen Hurts', 'Mark Andrews', 'Chris Olave', 'Najee Harris', 'Rhamondre Stevenson', 'Tee Higgins', 'DeVonta Smith', 'Travis Etienne', 'DK Metcalf', 'Breece Hall', 'Lamar Jackson', 'Jahmyr Gibbs', 'Joe Burrow', 'Joe Mixon', 'Deebo Samuel', 'Kenneth Walker', 'Calvin Ridley', 'Keenan Allen', 'Aaron Jones', 'Amari Cooper', 'Justin Fields', 'Justin Herbert', 'TJ Hockenson', 'George Kittle', 'DeAndre Hopkins', 'Terry McLaurin', 'Dameon Pierce', 'Miles Sanders', 'DJ Moore', 'JK Dobbins', 'Jerry Jeudy', 'Drake London', 'Cam Akers', 'Trevor Lawrence', 'Alexander Mattison', 'Christian Watson', 'Kyle Pitts', 'James Conner', "D'Andre Swift", 'Chris Godwin', 'Dallas Goedert', 'Darren Waller', 'Brandon Aiyuk', 'Mike Williams', 'Alvin Kamara', 'Tyler Loc

In [9]:
import pandas as pd
df = pd.read_csv('vorp2023.csv')
# get into the right format
df['Player'] = df['FirstName'] + ' ' + df['LastName']
players = df['Player'].tolist()
vorp = df.set_index('Player')['Y0_VORP'].to_dict()
player_position = df.set_index('Player')['Position'].to_dict()
ADP = df.set_index('Player')['RedraftHalfPPR'].to_dict()

In [12]:
ADP

{'Justin Jefferson': 1.5,
 'Christian McCaffrey': 2.2,
 "Ja'Marr Chase": 3.0,
 'Austin Ekeler': 4.0,
 'Travis Kelce': 5.7,
 'Tyreek Hill': 6.9,
 'Cooper Kupp': 7.5,
 'Bijan Robinson': 8.0,
 'Saquon Barkley': 8.6,
 'Stefon Diggs': 10.6,
 'Nick Chubb': 11.9,
 'AJ Brown': 12.9,
 'CeeDee Lamb': 13.2,
 'Patrick Mahomes': 14.8,
 'Derrick Henry': 15.2,
 'Davante Adams': 16.6,
 'Jonathan Taylor': 17.1,
 'Amon-Ra St Brown': 17.7,
 'Josh Jacobs': 19.9,
 'Josh Allen': 20.4,
 'Tony Pollard': 21.8,
 'Garrett Wilson': 22.3,
 'Jaylen Waddle': 23.5,
 'Jalen Hurts': 24.3,
 'Mark Andrews': 25.4,
 'Chris Olave': 26.0,
 'Najee Harris': 26.4,
 'Rhamondre Stevenson': 28.2,
 'Tee Higgins': 29.9,
 'DeVonta Smith': 30.4,
 'Travis Etienne': 31.1,
 'DK Metcalf': 32.8,
 'Breece Hall': 33.4,
 'Lamar Jackson': 34.1,
 'Jahmyr Gibbs': 35.2,
 'Joe Burrow': 35.7,
 'Joe Mixon': 37.4,
 'Deebo Samuel': 38.5,
 'Kenneth Walker': 39.7,
 'Calvin Ridley': 40.2,
 'Keenan Allen': 41.6,
 'Aaron Jones': 42.0,
 'Amari Cooper': 43.1